In [1]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


In [2]:
import numpy as np
#from matplotlib import pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
#import cv2
import os
#from sklearn import utils
import json
from keras.optimizers import Adam
import keras_tuner
from keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Cargamos los tensores y otros datos necesarios para el modelo
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/X_train.npy')
#X_test = np.load('/content/drive/MyDrive/Colab Notebooks/X_test.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/y_train.npy')
#y_test = np.load('/content/drive/MyDrive/Colab Notebooks/y_test.npy')
target_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]
num_targets = len(target_names)

In [4]:
# TERCER MODELO:
# Hyperparameter tuning

# Modelo basado en hyperparameter tuning a través de keras_tuner: en este modelo se pueden trabajar parámetros de regularización (L1 y L2),
# inicialización de pesos, funciones de activación, añadir sí o no dropout, etc.
# La elección de los hiperparámetros es libre, pero se aplicará al segundo modelo, es decir, al que se ha añadido más capas sobre el modelo base.

def build_model(hp):

    # Generamos un grid para filters
    kernel_choices = hp.Int("filters", min_value = 32, max_value = 128, step=32)

    # Generamos un grid para funciones de activación
    activations = hp.Choice("activation", values=["relu",
                                                  "sigmoid",
                                                  "softplus",
                                                  "softsign",
                                                  "tanh",
                                                  "selu",
                                                  "elu",
                                                  "exponential"])

    # Generamos un grid para diferentes inicializadores de pesos
    kernel_init_choices = hp.Choice("kernel_initializer", values = ["glorot_uniform",
                                                                    "truncated_normal",
                                                                    "random_uniform",
                                                                    "ones"])

    # Generamos un grid para diferentes regularizadores
    kernel_reg_choices = hp.Choice("kernel_regularizer", values = ["l1",
                                                                   "l2",
                                                                   "l1_l2"])

    #if kernel_reg_choices == "l1":
      #kernel_reg = regularizers.l1(0.001)
    #else:
      ##kernel_reg = regularizers.l2(0.001)

    # Generamos un grid para valores de Dropout
    dropout_values = hp.Float("rate", min_value = 0.1, max_value = 0.6, step = 0.1)

    # Definimos el modelo base
    model = Sequential()

    # Primera capa conv
    model.add(Conv2D(filters            = kernel_choices,
                     kernel_size        = (3, 3),
                     activation         = activations,
                     strides            = (1,1),
                     input_shape        = (150, 150, 3),
                     kernel_initializer = kernel_init_choices,
                     kernel_regularizer = kernel_reg_choices
                    ))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(rate = dropout_values))

    # Segunda capa conv
    model.add(Conv2D(filters            = kernel_choices,
                     kernel_size        = (3, 3),
                     activation         = activations,
                     strides            = (1,1),
                     kernel_initializer = kernel_init_choices,
                     kernel_regularizer = kernel_reg_choices))

    #model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(rate = dropout_values))

    # Tercera capa conv
    model.add(Conv2D(filters            = kernel_choices,
                     kernel_size        = (3, 3),
                     activation         = activations,
                     strides            = (1,1),
                     kernel_initializer = kernel_init_choices,
                     kernel_regularizer = kernel_reg_choices))

    #model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(rate = dropout_values))

    # Capas densas
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(rate = dropout_values))
    model.add(Dense(num_targets, activation='softmax'))

    # Compilar
    #learning_rate = hp.Choice("lr", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer='adam',    #Adam(learning_rate=learning_rate)
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model


In [6]:
%%time

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 20,
    directory='/content/drive/MyDrive/Colab Notebooks/',
    project_name='Keras_Tuner',
    overwrite = True)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# En vez de fit, usamos search
tuner.search(X_train, y_train,
             batch_size = 32,
             epochs = 15,
             validation_split=0.20,
             callbacks  = [early_stopping])


Trial 20 Complete [00h 03m 04s]
val_accuracy: 0.6730552315711975

Best val_accuracy So Far: 0.7361894249916077
Total elapsed time: 00h 16m 12s
CPU times: user 5min 48s, sys: 1min 15s, total: 7min 3s
Wall time: 16min 12s


In [7]:
tuner.results_summary() # Devuelve los mejores modelos

Results summary
Results in /content/drive/MyDrive/Colab Notebooks/Keras_Tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
filters: 64
activation: selu
kernel_initializer: glorot_uniform
kernel_regularizer: l2
rate: 0.2
Score: 0.7361894249916077

Trial 11 summary
Hyperparameters:
filters: 64
activation: relu
kernel_initializer: glorot_uniform
kernel_regularizer: l1
rate: 0.2
Score: 0.679819643497467

Trial 19 summary
Hyperparameters:
filters: 128
activation: softsign
kernel_initializer: random_uniform
kernel_regularizer: l1
rate: 0.1
Score: 0.6730552315711975

Trial 09 summary
Hyperparameters:
filters: 32
activation: relu
kernel_initializer: glorot_uniform
kernel_regularizer: l1
rate: 0.2
Score: 0.6257046461105347

Trial 10 summary
Hyperparameters:
filters: 96
activation: relu
kernel_initializer: glorot_uniform
kernel_regularizer: l1
rate: 0.5
Score: 0.6099210977554321

Trial 03 summary
Hyperparameters:
filters: 96
activation

In [8]:
# Obtener el mejor modelo
model3 = tuner.get_best_models(num_models=2)[0]
model3.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 18496)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       4,735,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           1,542 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,812,422 (18.36 MB)

 Trainable params: 4,812,422 (18.36 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
hist_model3 = model3.fit(X_train, y_train,
                  batch_size = 32,
                  epochs = 15,
                  validation_split = 0.20,
                  callbacks  = [early_stopping])

Epoch 1/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - accuracy: 0.8958 - loss: 0.5665 - val_accuracy: 0.7227 - val_loss: 1.3949
Epoch 2/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.9264 - loss: 0.4833 - val_accuracy: 0.6933 - val_loss: 1.4911
Epoch 3/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9366 - loss: 0.3961 - val_accuracy: 0.7148 - val_loss: 1.3993
Epoch 4/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.9630 - loss: 0.3167 - val_accuracy: 0.7035 - val_loss: 1.4393


In [10]:
model3.save('/content/drive/MyDrive/Colab Notebooks/model3.h5')
with open('/content/drive/MyDrive/Colab Notebooks/hist_model3.json', 'w') as f:
    json.dump(hist_model3.history, f)